# Análise de dados de mídia social

## Coletando os dados!

Para conseguir acesso a API do twitter para coleta de posts, é necessário seguir os passos conforme o site: https://developer.twitter.com/en/docs/basics/authentication/guides/access-tokens.html

In [ ]:
# Configurando a autenticação do pacote tweepy

# Carregando os pacotes necessários
from tweepy import OAuthHandler
from tweepy import API

# Autenticação da "consumer key"
consumer_key = "VGldO5N270dm4SBtiLSWi6UbM"
consumer_secret = "znpo5UgRmKqt9R1UuN2MonIw5TOkEVtvpuuZVSLvvwpRRhs9G0" 
auth = OAuthHandler(consumer_key, consumer_secret)

# Autenticação da "access key" 
access_token = "1087340053892919296-Ik0uNn63oStHrfQ8YUHoKAd5HanpUX"
access_token_secret = "VNzCDSqxD3nYdNUoLbIwPvJCOnBMZQwDBfmB8OufVFxZ1"
auth.set_access_token(access_token, access_token_secret)

# Configurando a API com a autenticação
api = API(auth)

In [ ]:
# Definindo as palavras ou hashtags que serão rastreadas
keywords_to_track = ['Trump']

In [ ]:
# Coletando posts do twitter através da API

from tweepy import Stream
from tweepy import StreamListener
import json
import time
import os
import sys

path = '/packages/Lib/site-packages'
sys.path.insert(0, os.getcwd() + path)

# Criando o SListener
class SListener(StreamListener):
    def __init__(self, api = None, fprefix = 'streamer'):
        self.api = api or API()
        self.counter = 0
        self.fprefix = fprefix
        self.output  = open('%s_%s.json' % (self.fprefix, time.strftime('%Y%m%d-%H%M%S')), 'w')

    def on_data(self, data):
        if 'in_reply_to_status' in data:
            self.on_status(data)
        elif 'delete' in data:
            delete = json.loads(data)['delete']['status']
            if self.on_delete(delete['id'], delete['user_id']) is False:
                return False
        elif 'limit' in data:
            if self.on_limit(json.loads(data)['limit']['track']) is False:
                return False
        elif 'warning' in data:
            warning = json.loads(data)['warnings']
            print("WARNING: %s" % warning['message'])
            return

    def on_status(self, status):
        self.output.write(status)
        self.counter += 1
        if self.counter >= 20000 # define quantos posts serão coletados, no caso são 20.000 posts
            self.output.close()
            self.output  = open('%s_%s.json' % (self.fprefix, time.strftime('%Y%m%d-%H%M%S')), 'w')
            self.counter = 0
        return

    def on_delete(self, status_id, user_id):
        print("Delete notice")
        return

    def on_limit(self, track):
        print("WARNING: Limitation notice received, tweets missed: %d" % track)
        return

    def on_error(self, status_code):
        print('Encountered error with status code:', status_code)
        return 

    def on_timeout(self):
        print("Timeout, sleeping for 60 seconds...")
        time.sleep(60)
        return 

# Criando uma instância para o SListener 
listen = SListener(api, 'Trump') # Inclui a palavra Trump no nome do arquivo

# Criando uma instância para Stream
stream = Stream(auth, listen)

# Coletando os posts do twitter!
print('A coleta dos posts começou! Obs: para pausar a coleta de posts do twitter através do stream, vá na aba "Files" do jupyter notebook, selecione o arquivo do jupyter notebook e clique em "Shutdown", isso é necessário para os códigos adiante funcionarem!')
stream.filter(track = keywords_to_track, languages=['en']) # o argumento "languages" define a língua dos posts que serão coletados no twitter

In [ ]:
# Carregando os pacotes para a transformação dos arquivo(s) .json para data frame do pandas
import glob
import pandas as pd
import numpy as np

In [ ]:
# Armazenando o nome dos arquivos .json que estão no diretório
files  = list(glob.iglob('*.json'))

In [ ]:
# Juntando os arquivos .json 
for f in files:
    fh = open(f, 'r', encoding = 'utf-8')
    tweets_json = fh.read().split("\n")

In [ ]:
# Removendo linhas vazias
tweets_json = list(filter(len, tweets_json))

In [ ]:
# Objeto que irá armazenar todos os posts do twitter 
tweets = []

In [ ]:
# Carregando pacote para ler os arquivos .json
import json

# Iterando através de cada tweet
for tweet in tweets_json:
    try:
        tweet_obj = json.loads(tweet)
    
        # Armazenando o nome do usuário
        tweet_obj['user-screen_name'] = tweet_obj['user']['screen_name']
    
        # Armazenando o nome do usuário que fez retweet e armazenando o texto 
        if 'retweeted_status' in tweet_obj:
            tweet_obj['retweeted_status-user-screen_name'] = tweet_obj['retweeted_status']['user']['screen_name']
            tweet_obj['retweeted_status-text'] = tweet_obj['retweeted_status']['text']
        
        # Armazenando citações
        if 'quoted_status' in tweet_obj:
            tweet_obj['quoted_status-text'] = tweet_obj['quoted_status']['text'] 
            tweet_obj['quoted_status-user-screen_name'] = tweet_obj['quoted_status']['user']['screen_name']
    
        tweets.append(tweet_obj)
    except:
        pass

In [ ]:
# Criando o data frame com pandas!
df_tweet = pd.DataFrame(tweets)
df_tweet

# Análise de sentimento das postagens

A análise de sentimento é uma ferramenta de classificação de texto que analisa as palavras e retorna um sentimento positivo, negativo ou neutro. 

In [ ]:
# Carregando o pacote SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Criando uma instância para a função SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

# Gerando scores de sentimento
sentiment = df_tweet['text'].apply(sid.polarity_scores)

In [ ]:
# Incluindo os scores de sentimento no data frame
s = []

for index, i in enumerate(sentiment):
    s.append(sentiment[index].get('compound'))

df_tweet['sentiment'] = s

In [ ]:
# Frequência de posts positivos e negativos
print("Posts positivos:" + str(sum(df_tweet['sentiment'] > 0.6)))
print("Posts negativos:" + str(sum(df_tweet['sentiment'] < -0.6)))

# Análise de rede

### Criando a rede de retweets

A mídia social é por natureza uma rede. As redes do Twitter se manifestam de várias maneiras, um dos tipos mais importantes de redes que aparecem no Twitter são as redes de retweets. Podemos representá-los como gráficos direcionados, com o usuário retweetando como a fonte e a pessoa retweetada como o alvo.

In [ ]:
# Carregando o pacote networkx
import networkx as nx

# Criando a rede de retweets a partir da edgelist do data frame
G_rt = nx.from_pandas_edgelist(
    df_tweet,
    source = 'user-screen_name', 
    target = 'retweeted_status-user-screen_name',
    create_using = nx.DiGraph())
    
# Visualizando o número de nós
print('Nós:', len(G_rt.nodes()))

# Visualizando o número de arestas
print('Arestas:', len(G_rt.edges()))

### Criando a rede de resposta

Enquanto a rede de retweets sinaliza concordância, a rede de resposta pode sinalizar discussão, deliberação e discordância. As propriedades de rede são as mesmas, no entanto: a rede é direcionada, a origem é a resposta e o destino é o usuário que está sendo respondido.

In [ ]:
# Criando a rede de resposta da edgelist
G_reply = nx.from_pandas_edgelist(
    df_tweet,
    source = 'user-screen_name', 
    target = 'in_reply_to_screen_name',
    create_using = nx.DiGraph())
    
# Visualizando o número de nós
print('Nós:', len(G_reply.nodes()))

# Visualizando o número de arestas
print('Arestas:', len(G_reply.edges()))

### Visualizando a rede de retweets

A visualização de redes de retweets é uma importante etapa da análise exploratória de dados pois nos permite inspecionar visualmente a estrutura da rede, entender se existe algum usuário que tenha influência desproporcional e se há diferentes esferas de conversação.

In [ ]:
import matplotlib.pyplot as plt

# Instância de tamanho dos nós
sizes = [x[1] for x in G_rt.degree()]

# Desenhando a rede de retweets!
nx.draw_networkx(G_rt, 
    with_labels = False, 
    node_size = sizes,
    width = 0.1, alpha = 0.5,
    arrowsize = 2, linewidths = 0)

plt.axis('off')
plt.figure(figsize=(25,25))
plt.show()

### Centralidade de grau

Centralidade de grau é uma métrica de importância de um nó para uma rede. Para redes direcionadas como as do Twitter, precisamos ter o cuidado de distinguir entre centralidade de grau de entrada e de saída, especialmente em redes de retweets. A centralidade de grau de entrada para redes de retweets sinaliza usuários que estão recebendo muitos retweets.

In [ ]:
# Centralidade de entrada para a rede de retweets 
rt_centrality = nx.in_degree_centrality(G_rt)

# Centralidade de entrada para a rede de respostas 
reply_centrality = nx.in_degree_centrality(G_reply)

# Armazenando as centralidades em um data frame
column_names = ['screen_name', 'degree_centrality']

rt = pd.DataFrame(list(rt_centrality.items()), columns = column_names)
reply = pd.DataFrame(list(reply_centrality.items()), columns = column_names)

# Visualizando os cinco primeiros nós com maior centralidade de grau na rede de retweets
print(rt.sort_values('degree_centrality', ascending = False).head())

# Visualizando os cinco primeiros nós com maior centralidade de grau na rede de respostas
print(reply.sort_values('degree_centrality', ascending = False).head())

### Centralidade de proximidade

A centralidade de proximidade para as redes de retweets e de respostas sinaliza que há usuários que fazem a ponte entre diferentes comunidades do Twitter. Essas comunidades podem estar ligadas por tópicos de interesse ou ideologia.

In [ ]:
# Centralidade de proximidade para a rede de retweets 
rt_centrality = nx.betweenness_centrality(G_rt)

# Centralidade de proximidade para a rede de respostas 
reply_centrality = nx.betweenness_centrality(G_reply)

# Armazenando as centralidades em um data frame
column_names = ['screen_name', 'betweenness_centrality']

rt = pd.DataFrame(list(rt_centrality.items()), columns = column_names)
reply = pd.DataFrame(list(reply_centrality.items()), columns = column_names)

# Visualizando os cinco primeiros nós com maior centralidade de proximidade na rede de retweets
print(rt.sort_values('betweenness_centrality', ascending = False).head())

# Visualizando os cinco primeiros nós com maior centralidade de proximidade na rede de respostas
print(reply.sort_values('betweenness_centrality', ascending = False).head())

### Ratio

"The Ratio", como é chamada, é uma medida de rede específica do Twitter e normalmente é usada para julgar a impopularidade de um tweet. É calculado tomando o número de respostas e dividindo pelo número de retweets, no caso desse estudo será utilizada a centralidade de grau de entrada.

In [ ]:
column_names = ['screen_name', 'degree']

degree_rt = pd.DataFrame(list(G_rt.in_degree()), columns = column_names)
degree_reply = pd.DataFrame(list(G_reply.in_degree()), columns = column_names)

# Juntando os dois data frames
ratio = degree_rt.merge(degree_reply, on = 'screen_name', suffixes = ('_rt', '_reply'))

# Calculando o ratio
ratio['ratio'] = ratio['degree_reply'] / ratio['degree_rt']

# Excluindo os tweets com menos de 5 retweets
ratio = ratio[ratio['degree_rt'] >= 5]

# Vizualizando os cinco maiores ratio
print(ratio.sort_values('ratio', ascending = False).head())

## Referências bibliográficas

Hutto, C.J. & Gilbert, E.E. (2014). VADER: A Parsimonious Rule-based Model for Sentiment Analysis of Social Media Text. Eighth International Conference on Weblogs and Social Media (ICWSM-14). Ann Arbor, MI, June 2014.